In [1]:
# Импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [3]:
# Загрузка данных
train_df = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv')

print(f"Размер датасета: {train_df.shape}")
print(f"\nПервые строки:")
print(train_df.head())
print(f"\nИнформация о данных:")
print(train_df.info())
print(f"\nПропущенные значения:")
print(train_df.isnull().sum())


Размер датасета: (517754, 14)

Первые строки:
   id road_type  num_lanes  curvature  speed_limit  lighting weather  \
0   0     urban          2       0.06           35  daylight   rainy   
1   1     urban          4       0.99           35  daylight   clear   
2   2     rural          4       0.63           70       dim   clear   
3   3   highway          4       0.07           35       dim   rainy   
4   4     rural          1       0.58           60  daylight   foggy   

   road_signs_present  public_road time_of_day  holiday  school_season  \
0               False         True   afternoon    False           True   
1                True        False     evening     True           True   
2               False         True     morning     True          False   
3                True         True     morning    False          False   
4               False        False     evening     True          False   

   num_reported_accidents  accident_risk  
0                       1        

In [4]:
# Подготовка данных для нейросети

# Разделение на признаки и целевую переменную
X = train_df.drop(['id', 'accident_risk'], axis=1)
y = train_df['accident_risk'].values

print("Типы признаков:")
print(X.dtypes)

# Определяем категориальные и числовые признаки
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nКатегориальные признаки: {categorical_features}")
print(f"Числовые признаки: {numerical_features}")

# Кодирование категориальных признаков с помощью Label Encoding
X_encoded = X.copy()
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le

print(f"\nРазмер после кодирования: {X_encoded.shape}")
print(f"Количество признаков: {X_encoded.shape[1]}")

Типы признаков:
road_type                  object
num_lanes                   int64
curvature                 float64
speed_limit                 int64
lighting                   object
weather                    object
road_signs_present           bool
public_road                  bool
time_of_day                object
holiday                      bool
school_season                bool
num_reported_accidents      int64
dtype: object

Категориальные признаки: ['road_type', 'lighting', 'weather', 'road_signs_present', 'public_road', 'time_of_day', 'holiday', 'school_season']
Числовые признаки: ['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents']

Размер после кодирования: (517754, 12)
Количество признаков: 12


In [5]:
# Нормализация числовых признаков
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[numerical_features] = scaler.fit_transform(X_encoded[numerical_features])

print("Данные после нормализации:")
print(X_scaled.head())
print(f"\nСтатистика числовых признаков после нормализации:")
print(X_scaled[numerical_features].describe())

Данные после нормализации:
   road_type  num_lanes  curvature  speed_limit  lighting  weather  \
0          2  -0.438680  -1.572918    -0.703840         0        2   
1          2   1.346344   1.839137    -0.703840         0        0   
2          1   1.346344   0.518342     1.512963         1        0   
3          0   1.346344  -1.536229    -0.703840         1        2   
4          1  -1.331192   0.334898     0.879591         0        1   

   road_signs_present  public_road  time_of_day  holiday  school_season  \
0                   0            1            0        0              1   
1                   1            0            1        1              1   
2                   0            1            2        1              0   
3                   1            1            2        0              0   
4                   0            0            1        1              0   

   num_reported_accidents  
0               -0.209797  
1               -1.325918  
2                

In [6]:
# Разделение на train и validation
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Train размер: {X_train.shape}")
print(f"Validation размер: {X_val.shape}")

# Конвертация в numpy arrays
X_train_np = X_train.values.astype(np.float32)
X_val_np = X_val.values.astype(np.float32)
y_train_np = y_train.astype(np.float32).reshape(-1, 1)
y_val_np = y_val.astype(np.float32).reshape(-1, 1)


Train размер: (414203, 12)
Validation размер: (103551, 12)


In [7]:
# Определение архитектуры нейросети
class AccidentRiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super(AccidentRiskPredictor, self).__init__()
        
        self.network = nn.Sequential(
            # Input layer 
            nn.Linear(input_dim, 512), 
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.1),  
            
            nn.Linear(512, 256), 
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(256, 128), 
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.05),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            # Без dropout на последних слоях
            
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

# Создание модели
input_dim = X_train_np.shape[1]
model = AccidentRiskPredictor(input_dim).to(device)

print(f"\nКоличество параметров: {sum(p.numel() for p in model.parameters())}")



Количество параметров: 183777


In [8]:
# Настройка обучения
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=15, factor=0.5, verbose=True)

# Параметры обучения
batch_size = 512
num_epochs = 200
best_val_loss = float('inf')
patience = 50
patience_counter = 0

# Создание DataLoader для батчевого обучения
train_dataset = TensorDataset(
    torch.FloatTensor(X_train_np), 
    torch.FloatTensor(y_train_np)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Для валидации
X_val_tensor = torch.FloatTensor(X_val_np).to(device)
y_val_tensor = torch.FloatTensor(y_val_np).to(device)

print(f"Параметры обучения:")
print(f"Batch size: {batch_size}")
print(f"Количество эпох: {num_epochs}")
print(f"Learning rate: 0.001")
print(f"Оптимизатор: Adam")
print(f"Функция потерь: MSE")
print(f"Device: {device}")


Параметры обучения:
Batch size: 512
Количество эпох: 200
Learning rate: 0.001
Оптимизатор: Adam
Функция потерь: MSE
Device: cuda


In [9]:
# ОБУЧЕНИЕ НЕЙРОСЕТИ 
train_losses = []
val_losses = []

print("ОБУЧЕНИЕ НЕЙРОСЕТИ")
print("Начало обучения...\n")

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    
    for batch_X, batch_y in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * batch_X.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())
    
    # Learning rate scheduling
    scheduler.step(val_loss)
    
    # Вывод прогресса
    if (epoch + 1) % 10 == 0:
        train_rmse = np.sqrt(train_loss)
        val_rmse = np.sqrt(val_loss.item())
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'  Train Loss: {train_loss:.6f} (RMSE: {train_rmse:.6f})')
        print(f'  Val Loss: {val_loss.item():.6f} (RMSE: {val_rmse:.6f})')
        print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}\n')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print(f'Early stopping на эпохе {epoch+1}')
        model.load_state_dict(best_model_state)
        break

print("Обучение нейросети завершено!")
print(f"Лучший Val Loss: {best_val_loss:.6f} (RMSE: {np.sqrt(best_val_loss.item()):.6f})")

ОБУЧЕНИЕ НЕЙРОСЕТИ
Начало обучения...



Epoch 10/200: 100%|██████████| 809/809 [00:04<00:00, 172.24it/s]


Epoch [10/200]
  Train Loss: 0.003391 (RMSE: 0.058233)
  Val Loss: 0.003435 (RMSE: 0.058611)
  LR: 0.001000



Epoch 20/200: 100%|██████████| 809/809 [00:05<00:00, 152.52it/s]


Epoch [20/200]
  Train Loss: 0.003371 (RMSE: 0.058058)
  Val Loss: 0.003294 (RMSE: 0.057391)
  LR: 0.001000



...

Epoch [180/200]
  Train Loss: 0.003204 (RMSE: 0.056603)
  Val Loss: 0.003213 (RMSE: 0.056683)
  LR: 0.000016



Epoch 190/200: 100%|██████████| 809/809 [00:05<00:00, 159.22it/s]


Epoch [190/200]
  Train Loss: 0.003198 (RMSE: 0.056548)
  Val Loss: 0.003220 (RMSE: 0.056748)
  LR: 0.000016



Epoch 200/200: 100%|██████████| 809/809 [00:05<00:00, 159.31it/s]

Epoch [200/200]
  Train Loss: 0.003201 (RMSE: 0.056580)
  Val Loss: 0.003205 (RMSE: 0.056614)
  LR: 0.000016

Обучение нейросети завершено!
Лучший Val Loss: 0.003198 (RMSE: 0.056548)


In [10]:
# ОБУЧЕНИЕ XGBOOST 
import xgboost as xgb

print("ОБУЧЕНИЕ XGBOOST МОДЕЛИ")

# Используем уже закодированные данные X_encoded (до нормализации)
X_train_xgb, X_val_xgb, y_train_xgb, y_val_xgb = train_test_split(
    X_encoded, y, 
    test_size=0.2, 
    random_state=42
)

# Обучение XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=1000,
    max_depth=8,           
    learning_rate=0.05,   
    
    # Умеренная регуляризация
    reg_alpha=0.01,
    reg_lambda=0.1,
    gamma=0.01,
    
    # Оптимальное сэмплирование
    subsample=0.9,
    colsample_bytree=0.9,
    
    # Простые настройки
    min_child_weight=1,
    tree_method='hist',
    
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=50
)

print("\nНачало обучения XGBoost...")
xgb_model.fit(
    X_train_xgb, y_train_xgb,
    eval_set=[(X_train_xgb, y_train_xgb), (X_val_xgb, y_val_xgb)],
    verbose=50
)

# Предсказания XGBoost
y_train_pred_xgb = xgb_model.predict(X_train_xgb)
y_val_pred_xgb = xgb_model.predict(X_val_xgb)

# Метрики XGBoost
print("МЕТРИКИ XGBOOST:")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train_xgb, y_train_pred_xgb)):.6f}")
print(f"Val RMSE: {np.sqrt(mean_squared_error(y_val_xgb, y_val_pred_xgb)):.6f}")


ОБУЧЕНИЕ XGBOOST МОДЕЛИ

Начало обучения XGBoost...
[0]	validation_0-rmse:0.16119	validation_1-rmse:0.16092
[50]	validation_0-rmse:0.05911	validation_1-rmse:0.05961
[100]	validation_0-rmse:0.05561	validation_1-rmse:0.05628
[150]	validation_0-rmse:0.05547	validation_1-rmse:0.05619
[200]	validation_0-rmse:0.05544	validation_1-rmse:0.05619
[250]	validation_0-rmse:0.05544	validation_1-rmse:0.05619
[300]	validation_0-rmse:0.05544	validation_1-rmse:0.05619
[350]	validation_0-rmse:0.05544	validation_1-rmse:0.05618
[400]	validation_0-rmse:0.05544	validation_1-rmse:0.05618
[450]	validation_0-rmse:0.05544	validation_1-rmse:0.05618
[500]	validation_0-rmse:0.05544	validation_1-rmse:0.05618
[550]	validation_0-rmse:0.05544	validation_1-rmse:0.05618
[562]	validation_0-rmse:0.05544	validation_1-rmse:0.05618
МЕТРИКИ XGBOOST:
Train RMSE: 0.055439
Val RMSE: 0.056184


In [11]:
# ПРЕДСКАЗАНИЯ НЕЙРОСЕТИ НА XGB DATA 

# Нормализуем данные XGBoost для нейросети
X_train_xgb_scaled = X_train_xgb.copy()
X_val_xgb_scaled = X_val_xgb.copy()

X_train_xgb_scaled[numerical_features] = scaler.transform(X_train_xgb[numerical_features])
X_val_xgb_scaled[numerical_features] = scaler.transform(X_val_xgb[numerical_features])

# Предсказания нейросети
X_train_xgb_np = X_train_xgb_scaled.values.astype(np.float32)
X_val_xgb_np = X_val_xgb_scaled.values.astype(np.float32)

model.eval()
with torch.no_grad():
    X_train_xgb_tensor = torch.FloatTensor(X_train_xgb_np).to(device)
    X_val_xgb_tensor = torch.FloatTensor(X_val_xgb_np).to(device)
    
    y_train_pred_nn = model(X_train_xgb_tensor).cpu().numpy().flatten()
    y_val_pred_nn = model(X_val_xgb_tensor).cpu().numpy().flatten()

print("МЕТРИКИ НЕЙРОСЕТИ:")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train_xgb, y_train_pred_nn)):.6f}")
print(f"Val RMSE: {np.sqrt(mean_squared_error(y_val_xgb, y_val_pred_nn)):.6f}")

МЕТРИКИ НЕЙРОСЕТИ:
Train RMSE: 0.056126
Val RMSE: 0.056614


In [12]:
# СОЗДАНИЕ АНСАМБЛЯ 

# Попробуем разные веса
weights_to_test = [
    (0.5, 0.5),   # Равные веса
    (0.6, 0.4),   # Больше XGBoost
    (0.7, 0.3),   # Еще больше XGBoost
    (0.4, 0.6),   # Больше NN
    (0.55, 0.45), # Немного больше XGBoost
    (0.65, 0.35),
    (0.45, 0.55),
]

print("ПОИСК ОПТИМАЛЬНЫХ ВЕСОВ АНСАМБЛЯ")

best_rmse = float('inf')
best_weights = None

for w_xgb, w_nn in weights_to_test:
    # Ансамбль на validation
    ensemble_val_pred = w_xgb * y_val_pred_xgb + w_nn * y_val_pred_nn
    ensemble_rmse = np.sqrt(mean_squared_error(y_val_xgb, ensemble_val_pred))
    
    print(f"Веса: XGB={w_xgb:.2f}, NN={w_nn:.2f} → RMSE: {ensemble_rmse:.6f}")
    
    if ensemble_rmse < best_rmse:
        best_rmse = ensemble_rmse
        best_weights = (w_xgb, w_nn)

print(f"ЛУЧШИЕ ВЕСА: XGB={best_weights[0]:.2f}, NN={best_weights[1]:.2f}")
print(f"ЛУЧШИЙ ENSEMBLE VAL RMSE: {best_rmse:.6f}")

# Создаем финальный ансамбль с лучшими весами
w_xgb_final, w_nn_final = best_weights
y_train_pred_ensemble = w_xgb_final * y_train_pred_xgb + w_nn_final * y_train_pred_nn
y_val_pred_ensemble = w_xgb_final * y_val_pred_xgb + w_nn_final * y_val_pred_nn

print("ФИНАЛЬНЫЕ МЕТРИКИ АНСАМБЛЯ")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train_xgb, y_train_pred_ensemble)):.6f}")
print(f"Val RMSE:   {np.sqrt(mean_squared_error(y_val_xgb, y_val_pred_ensemble)):.6f}")
print(f"Val R2:     {r2_score(y_val_xgb, y_val_pred_ensemble):.6f}")

ПОИСК ОПТИМАЛЬНЫХ ВЕСОВ АНСАМБЛЯ
Веса: XGB=0.50, NN=0.50 → RMSE: 0.056265
Веса: XGB=0.60, NN=0.40 → RMSE: 0.056228
Веса: XGB=0.70, NN=0.30 → RMSE: 0.056201
Веса: XGB=0.40, NN=0.60 → RMSE: 0.056314
Веса: XGB=0.55, NN=0.45 → RMSE: 0.056245
Веса: XGB=0.65, NN=0.35 → RMSE: 0.056213
Веса: XGB=0.45, NN=0.55 → RMSE: 0.056288
ЛУЧШИЕ ВЕСА: XGB=0.70, NN=0.30
ЛУЧШИЙ ENSEMBLE VAL RMSE: 0.056201
ФИНАЛЬНЫЕ МЕТРИКИ АНСАМБЛЯ
Train RMSE: 0.055528
Val RMSE:   0.056201
Val R2:     0.885612


In [13]:
# СРАВНИТЕЛЬНАЯ ТАБЛИЦА 

comparison = pd.DataFrame({
    'Модель': ['XGBoost', 'Нейросеть', 'Ансамбль'],
    'Train RMSE': [
        np.sqrt(mean_squared_error(y_train_xgb, y_train_pred_xgb)),
        np.sqrt(mean_squared_error(y_train_xgb, y_train_pred_nn)),
        np.sqrt(mean_squared_error(y_train_xgb, y_train_pred_ensemble))
    ],
    'Val RMSE': [
        np.sqrt(mean_squared_error(y_val_xgb, y_val_pred_xgb)),
        np.sqrt(mean_squared_error(y_val_xgb, y_val_pred_nn)),
        np.sqrt(mean_squared_error(y_val_xgb, y_val_pred_ensemble))
    ]
})

print("СРАВНЕНИЕ МОДЕЛЕЙ")
print(comparison.to_string(index=False))

# Улучшение ансамбля относительно лучшей модели
xgb_rmse = comparison.loc[0, 'Val RMSE']
ensemble_rmse = comparison.loc[2, 'Val RMSE']
improvement = ((xgb_rmse - ensemble_rmse) / xgb_rmse) * 100

print(f"\nУлучшение ансамбля относительно XGBoost: {improvement:.3f}%")
print(f"Веса: XGBoost={w_xgb_final:.1%}, Нейросеть={w_nn_final:.1%}")


СРАВНЕНИЕ МОДЕЛЕЙ
   Модель  Train RMSE  Val RMSE
  XGBoost    0.055439  0.056184
Нейросеть    0.056126  0.056614
 Ансамбль    0.055528  0.056201

Улучшение ансамбля относительно XGBoost: -0.029%
Веса: XGBoost=70.0%, Нейросеть=30.0%


In [14]:
# ПРЕДСКАЗАНИЕ НА TEST ДАННЫХ (АНСАМБЛЬ) 
try:
    test_df = pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv')
    print(f"Тестовый набор загружен: {test_df.shape}")
    
    # Сохраняем ID
    test_ids = test_df['id']
    
    # Подготовка тестовых данных
    X_test = test_df.drop(['id', 'accident_risk'], axis=1, errors='ignore')
    
    #  ПРЕДСКАЗАНИЕ XGBOOST 
    X_test_encoded = X_test.copy()
    for col in categorical_features:
        if col in X_test_encoded.columns:
            X_test_encoded[col] = label_encoders[col].transform(X_test_encoded[col].astype(str))
    
    test_pred_xgb = xgb_model.predict(X_test_encoded)
    
    #  ПРЕДСКАЗАНИЕ НЕЙРОСЕТИ 
    X_test_scaled = X_test_encoded.copy()
    X_test_scaled[numerical_features] = scaler.transform(X_test_encoded[numerical_features])
    
    X_test_np = X_test_scaled.values.astype(np.float32)
    X_test_tensor = torch.FloatTensor(X_test_np).to(device)
    
    model.eval()
    with torch.no_grad():
        test_pred_nn = model(X_test_tensor).cpu().numpy().flatten()
    
    # АНСАМБЛЬ 
    test_pred_ensemble = w_xgb_final * test_pred_xgb + w_nn_final * test_pred_nn
    
    # Создание submission файла
    submission = pd.DataFrame({
        'id': test_ids,
        'accident_risk': test_pred_ensemble
    })
    
    submission.to_csv('submission.csv', index=False)
    
    print("АНСАМБЛЬ: submission.csv создан!")
    print(f"\nРазмер: {submission.shape}")
    print(f"Веса: XGBoost={w_xgb_final:.1%}, Нейросеть={w_nn_final:.1%}")
    print(f"\nСтатистика предсказаний:")
    print(submission['accident_risk'].describe())
    
except FileNotFoundError:
    print("Файл test.csv не найден.")


Тестовый набор загружен: (172585, 13)
АНСАМБЛЬ: submission.csv создан!

Размер: (172585, 2)
Веса: XGBoost=70.0%, Нейросеть=30.0%

Статистика предсказаний:
count    172585.000000
mean          0.352444
std           0.156244
min           0.027695
25%           0.241534
50%           0.337587
75%           0.454613
max           0.868336
Name: accident_risk, dtype: float64
